In [1]:
# CELL 1: Importy i konfiguracja
import os
import re
import numpy as np
import pandas as pd
from datetime import datetime
import joblib
import csv
import gc  

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# Konfiguracja
SEED = 42
tf.keras.utils.set_random_seed(SEED)
np.random.seed(SEED)
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# CELL 2 (NOWA, SOLIDNA WERSJA): Wczytywanie i mapowanie po indeksach

import pandas as pd
import numpy as np

PATH = 'Data_state_LSTM_predicted_full_v4_FINAL.csv'

# Krok 1: Wczytaj plik, całkowicie ignorując nagłówek
try:
    df = pd.read_csv(PATH, sep=';', encoding='utf-8-sig', header=None, skiprows=1, low_memory=False)
    print(f"Wczytano {len(df)} wierszy z pliku: {PATH}")
except Exception as e:
    raise RuntimeError(f"Nie udało się wczytać pliku. Upewnij się, że istnieje i ma separator ';'. Błąd: {e}")

# Krok 2: Zdefiniuj mapowanie INDEKSÓW kolumn na CZYTELNE NAZWY
# To jest klucz do sukcesu - bazujemy na stałej pozycji kolumny, a nie na jej "brudnej" nazwie.
# Sprawdzamy, które indeksy odpowiadają którym danym na podstawie struktury pliku
# z poprzedniego notebooka.
# UWAGA: Indeksy liczymy od 0.
column_index_map = {
    0: 'SaleId',
    3: 'Title',
    4: 'Description',  # <-- Opis jest na 5. pozycji (indeks 4)
    5: 'Area',
    6: 'Price',
    11: 'NumberOfRooms',
    12: 'BuiltYear',
    14: 'BuildingType',
    16: 'OfferFrom',
    17: 'Floor',
    18: 'Floors',
    19: 'TypeOfMarket',
    # Kolumny z poprzednich modeli (zwykle na końcu)
    54: 'Predicted_Loc',
    55: 'Predict_State'
}

# Filtrujemy mapę, aby użyć tylko indeksów, które istnieją w DataFrame
valid_index_map = {idx: name for idx, name in column_index_map.items() if idx < df.shape[1]}

# Wybieramy tylko interesujące nas kolumny i od razu nadajemy im poprawne nazwy
df_clean = df[list(valid_index_map.keys())].copy()
df_clean.columns = list(valid_index_map.values())

print(f"\nWybrano i przemianowano {len(df_clean.columns)} kluczowych kolumn.")

# --- Podstawowe czyszczenie ---
df_clean['Price'] = pd.to_numeric(df_clean['Price'], errors='coerce')
df_clean['Area'] = pd.to_numeric(df_clean['Area'], errors='coerce')
df_clean = df_clean.dropna(subset=['Price', 'Area'])
df_clean = df_clean[df_clean['Price'] > 1000]
print(f"\nDane po podstawowym czyszczeniu (usunięto braki w Price/Area): {df_clean.shape}")
display(df_clean.head())

Wczytano 1467262 wierszy z pliku: Data_state_LSTM_predicted_full_v4_FINAL.csv

Wybrano i przemianowano 14 kluczowych kolumn.

Dane po podstawowym czyszczeniu (usunięto braki w Price/Area): (1260266, 14)


,SaleId,Title,Description,Area,Price,NumberOfRooms,BuiltYear,BuildingType,OfferFrom,Floor,Floors,TypeOfMarket,Predicted_Loc,Predict_State
0,99,Sprzedam mieszkanie na parterze 64.8m2 Białyst...,Sprzedam mieszkanie na parterze 64.8m2 w 3-pię...,64.80,"540,000.00",3,NaN,Blok,Osoba prywatna,0,NaN,Wtórny,Białystok -> ? -> ? -> ?,FOR_RENOVATION
1,115,"Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki",SPRZEDAŻ WYŁĄCZNIE BEZPOŚREDNIA. Agencjom nier...,51.00,"540,000.00",3,2013,NaN,Osoba prywatna,0,NaN,Wtórny,Białystok -> ? -> ? -> ?,AFTER_RENOVATION
2,140,Mieszkanie trzypokojowe na sprzedaż,***Oferta bez prowizji biura i podatku PCC!***...,67.62,"544,000.00",3,2023,Apartamentowiec,Agencja,0,1,Wtórny,Białystok -> ? -> ? -> ?,GOOD
3,145,3 Pokoje- 48M2-Osiedle Dziesięciny,Przedstawiamy na sprzedaż 3 pokojowe mieszkani...,48.00,"459,000.00",3,NaN,Blok,Agencja,3,NaN,Wtórny,Białystok -> ? -> ? -> ?,AFTER_RENOVATION
4,159,"Mieszkanie, 87 m², Białystok","Przestronne, jasne mieszkanie na zamkniętym os...",87.00,"779,000.00",4,2005,Blok,Osoba prywatna,1,NaN,Wtórny,Białystok -> ? -> ? -> ?,AFTER_RENOVATION


In [3]:
# CELL 3 (uproszczony): Inżynieria Cech

df_proc = df_clean.copy() # Pracujemy na czystej ramce danych z komórki 2

# --- Inżynieria Cech ---
# Konwersja numerycznych
num_cols_to_convert = ['NumberOfRooms','Floor','Floors','BuiltYear']
for c in num_cols_to_convert:
    if c in df_proc.columns:
        df_proc[c] = pd.to_numeric(df_proc[c], errors='coerce')

# BuiltYear -> BuildingAge
if 'BuiltYear' in df_proc.columns:
    by = df_proc['BuiltYear']
    median_year = by.dropna().median() if not by.dropna().empty else 2000
    by = by.fillna(median_year).clip(1800, datetime.now().year + 1)
    df_proc['BuildingAge'] = (datetime.now().year - by).astype(int)
else:
    df_proc['BuildingAge'] = 60

# Usuwanie outlierów z ceny
q_low = df_proc['Price'].quantile(0.01)
q_high = df_proc['Price'].quantile(0.99)
df_proc = df_proc[(df_proc['Price'] >= q_low) & (df_proc['Price'] <= q_high)]
print(f"\nDane po usunięciu 2% skrajnych cen (outlierów): {df_proc.shape}")

# Czyszczenie tekstu
def clean_text(s: str) -> str:
    s = (s or "").lower()
    patterns = [r'oferta nie stanowi.*?oferty w rozumieniu kodeksu cywilnego', r'prosz[ąa] o kontakt.*', r'tylko u nas.*', r'nie pobieramy prowizji.*']
    for p in patterns: s = re.sub(p, ' ', s, flags=re.IGNORECASE)
    s = re.sub(r'[^a-zA-Ząćęłńóśźż\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s
df_proc['Description'] = df_proc['Description'].fillna('').astype(str).apply(clean_text)

# --- Definicja finalnych list cech ---
numeric_features = [c for c in ['Area','NumberOfRooms','Floor','Floors','BuildingAge'] if c in df_proc.columns]
categorical_features = [c for c in ['Predict_State','Predicted_Loc','BuildingType','TypeOfMarket','OfferFrom'] if c in df_proc.columns] # Usunąłem 'Type' i 'OwnerType' dla uproszczenia, jeśli ich nie ma
text_feature = 'Description'

# Wypełnianie braków
for c in numeric_features:
    df_proc[c].fillna(df_proc[c].median(), inplace=True)
for c in categorical_features:
    df_proc[c] = df_proc[c].astype(str).fillna('unknown').replace({'nan':'unknown','None':'unknown'})

print("\nUżyte cechy numeryczne:", numeric_features)
print("Użyte cechy kategoryczne:", categorical_features)
print("Użyta cecha tekstowa:", text_feature)

# Logarytmowanie ceny
df_proc['Price_log'] = np.log1p(df_proc['Price'])


Dane po usunięciu 2% skrajnych cen (outlierów): (1235203, 15)

Użyte cechy numeryczne: ['Area', 'NumberOfRooms', 'Floor', 'Floors', 'BuildingAge']
Użyte cechy kategoryczne: ['Predict_State', 'Predicted_Loc', 'BuildingType', 'TypeOfMarket', 'OfferFrom']
Użyta cecha tekstowa: Description


In [4]:
# CELL 4 (poprawiony): Podział na zbiory i tworzenie WYDAJNYCH tf.data.Dataset

features = numeric_features + categorical_features + [text_feature]
target = 'Price_log'

train_df, val_df = train_test_split(df_proc, test_size=0.2, random_state=SEED)

del df_proc; gc.collect()

print(f"Zbiór treningowy: {train_df.shape}")
print(f"Zbiór walidacyjny: {val_df.shape}")

# Funkcja tworzy teraz dataset, który zwraca (słownik_cech, etykieta)
def df_to_dataset(dataframe, shuffle=True, batch_size=256):
    df = dataframe.copy()
    labels = df.pop(target).values
    # Tworzymy słownik z tablic numpy, co jest bardziej wydajne dla TF
    features_dict = {col: df[col].values for col in features}
    ds = tf.data.Dataset.from_tensor_slices((features_dict, labels))
    if shuffle:
        # Tasujemy cały zbiór przed batchowaniem - dla zbiorów mieszczących się w RAM jest to OK
        ds = ds.shuffle(buffer_size=len(dataframe), seed=SEED)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

print("\nTworzenie datasetów...")
train_ds = df_to_dataset(train_df)
val_ds = df_to_dataset(val_df, shuffle=False)

# Dataset do adaptacji: tylko cechy, bez etykiet
train_features_df = train_df[features]
adapt_ds = tf.data.Dataset.from_tensor_slices(dict(train_features_df)).batch(256)
print("Datasety gotowe.")

Zbiór treningowy: (988162, 16)
Zbiór walidacyjny: (247041, 16)

Tworzenie datasetów...
Datasety gotowe.


In [5]:
# CELL 5 (poprawiony): Budowa modelu z WYDAJNĄ PAMIĘCIOWO adaptacją

# --- 1. Przygotowanie warstw preprocessingu ---
inputs = {}
encoded_features = []

# A. Cechy numeryczne
print("Adaptacja warstw numerycznych...")
for feature_name in numeric_features:
    inputs[feature_name] = keras.Input(shape=(1,), name=feature_name, dtype=tf.float32)
    normalizer = layers.Normalization(axis=-1) # Jawnie określamy oś
    
    # *** KLUCZOWA POPRAWKA: Dodajemy tf.expand_dims, aby nadać danym poprawny kształt ***
    feature_ds = adapt_ds.map(lambda x: tf.expand_dims(x[feature_name], axis=-1))
    
    normalizer.adapt(feature_ds)
    encoded = normalizer(inputs[feature_name])
    encoded_features.append(encoded)

# B. Cechy kategoryczne (One-Hot)
print("Adaptacja warstw kategorycznych...")
for feature_name in categorical_features:
    inputs[feature_name] = keras.Input(shape=(1,), name=feature_name, dtype=tf.string)
    feature_ds = adapt_ds.map(lambda x: x[feature_name])
    lookup = layers.StringLookup(output_mode='one_hot')
    lookup.adapt(feature_ds)
    encoded = lookup(inputs[feature_name])
    encoded_features.append(encoded)

# C. Cecha tekstowa (Description - Multi-Hot)
print("Adaptacja warstwy tekstowej (to może chwilę potrwać)...")
inputs[text_feature] = keras.Input(shape=(1,), name=text_feature, dtype=tf.string)
text_ds = adapt_ds.map(lambda x: x[text_feature])
text_vectorizer = layers.TextVectorization(max_tokens=2000, output_mode='multi_hot')
text_vectorizer.adapt(text_ds)
encoded_text = text_vectorizer(inputs[text_feature])
encoded_features.append(encoded_text)

print("Adaptacja zakończona.")

# --- 2. Połączenie wszystkich przetworzonych cech ---
all_features = layers.Concatenate()(encoded_features)

# --- 3. Głowica regresyjna (Deep part) ---
x = layers.Dense(256, activation="relu")(all_features)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
output = layers.Dense(1, name="price_log")(x)

model = keras.Model(inputs, output)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError(name="rmse")]
)

model.summary()

Adaptacja warstw numerycznych...
Adaptacja warstw kategorycznych...
Adaptacja warstwy tekstowej (to może chwilę potrwać)...
Adaptacja zakończona.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Area (InputLayer)             │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ NumberOfRooms (InputLayer)    │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Floor (InputLayer)            │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Floors (InputLayer)           │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ BuildingAge (InputLayer)      │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Predict_State (InputLayer)    │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Predicted_Loc (InputLayer)    │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ BuildingType (InputLayer)     │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ TypeOfMarket (InputLayer)     │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ OfferFrom (InputLayer)        │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Description (InputLayer)      │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization (Normalization) │ (None, 1)                 │               3 │ Area[0][0]                 │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization_1               │ (None, 1)                 │               3 │ NumberOfRooms[0][0]        │
│ (Normalization)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization_2               │ (None, 1)                 │               3 │ Floor[0][0]                │
│ (Normalization)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization_3               │ (None, 1)                 │               3 │ Floors[0][0]               │
│ (Normalization)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ normalization_4               │ (None, 1)                 │               

 Total params: 2,350,352 (8.97 MB)

 Trainable params: 2,350,337 (8.97 MB)

 Non-trainable params: 15 (80.00 B)

In [6]:
# CELL 6: Trening modelu
es = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
rlr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)
csv_logger = keras.callbacks.CSVLogger('training_log_price_v7.csv')

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[es, rlr, csv_logger]
)

Epoch 1/50
3861/3861 ━━━━━━━━━━━━━━━━━━━━ 238s 53ms/step - loss: 5.7018 - rmse: 2.1656 - val_loss: 0.1582 - val_rmse: 0.3977 - learning_rate: 0.0010
Epoch 2/50
3861/3861 ━━━━━━━━━━━━━━━━━━━━ 263s 60ms/step - loss: 1.4056 - rmse: 1.1851 - val_loss: 0.2282 - val_rmse: 0.4777 - learning_rate: 0.0010
Epoch 3/50
3861/3861 ━━━━━━━━━━━━━━━━━━━━ 274s 63ms/step - loss: 0.8945 - rmse: 0.9448 - val_loss: 0.0936 - val_rmse: 0.3060 - learning_rate: 0.0010
Epoch 4/50
3861/3861 ━━━━━━━━━━━━━━━━━━━━ 287s 66ms/step - loss: 0.5261 - rmse: 0.7248 - val_loss: 0.0993 - val_rmse: 0.3151 - learning_rate: 0.0010
Epoch 5/50
3861/3861 ━━━━━━━━━━━━━━━━━━━━ 288s 66ms/step - loss: 0.3127 - rmse: 0.5588 - val_loss: 0.0771 - val_rmse: 0.2777 - learning_rate: 0.0010
Epoch 6/50
3861/3861 ━━━━━━━━━━━━━━━━━━━━ 294s 68ms/step - loss: 0.1729 - rmse: 0.4155 - val_loss: 0.0726 - val_rmse: 0.2695 - learning_rate: 0.0010
Epoch 7/50
3861/3861 ━━━━━━━━━━━━━━━━━━━━ 302s 70ms/step - loss: 0.1018 - rmse: 0.3189 - val_loss: 0.0582 

In [7]:
# CELL 7: Ocena, zapis modelu i finalna analiza z filtrowaniem predykcji

print("\n--- Ocena finalnego modelu na zbiorze walidacyjnym ---")
results = model.evaluate(val_ds, verbose=0)
print(f"Validation RMSE (on log scale): {results[1]:.4f}")

# Zapis modelu
MODEL_SAVE_PATH = 'price_regressor_v7_with_preprocessing.keras'
model.save(MODEL_SAVE_PATH)
print(f"\nModel z warstwami preprocessingu zapisany w: {MODEL_SAVE_PATH}")


# --- Pełna predykcja na zbiorze walidacyjnym do obliczenia MAPE i MedAPE ---
print("\nObliczanie efektywności procentowej...")
reloaded_model = keras.models.load_model(MODEL_SAVE_PATH)
val_labels_log = val_df[target]
val_features_dict = {name: np.array(data) for name, data in val_df[features].items()}
val_ds_full = tf.data.Dataset.from_tensor_slices(val_features_dict).batch(2048)
predicted_price_log_full = reloaded_model.predict(val_ds_full, verbose=0)

true_prices = np.expm1(val_labels_log.values)
predicted_prices = np.expm1(predicted_price_log_full.flatten())

absolute_percentage_errors = np.abs((true_prices - predicted_prices) / true_prices)
mape = np.mean(absolute_percentage_errors) * 100
median_ape = np.median(absolute_percentage_errors) * 100

print("\n" + "="*45)
print("--- Efektywność Procentowa Nowego Modelu ---")
print("="*45)
print(f"Średni Absolutny Błąd Procentowy (MAPE): {mape:.2f}%")
print(f"Mediana Absolutnego Błędu Procentowego: {median_ape:.2f}%")
print("="*45)
print(f"Interpretacja: Dla połowy mieszkań błąd predykcji jest mniejszy niż {median_ape:.2f}%.")


# --- NOWOŚĆ: Test predykcji z filtrowaniem pewności ---
print("\n--- Test predykcji z filtrowaniem pewności ---")

# Określamy "rozsądny" przedział cenowy na podstawie danych treningowych
price_bounds = train_df['Price'].quantile([0.05, 0.95]).to_dict()
print(f"Uznajemy predykcje za 'pewne', jeśli mieszczą się w przedziale: {price_bounds[0.05]:,.0f} - {price_bounds[0.95]:,.0f} PLN")

# Bierzemy 10 losowych próbek do testu
sample_df = val_df.sample(10, random_state=SEED)
sample_input = {name: np.array(data) for name, data in sample_df[features].items()}

# Predykcja
predicted_price_log_sample = reloaded_model.predict(sample_input)
predicted_price_sample = np.expm1(predicted_price_log_sample.flatten())

# Przygotowanie tabeli wyników
comparison = pd.DataFrame({
    'Prawdziwa Cena': np.expm1(sample_df[target]),
    'Przewidziana Cena': predicted_price_sample,
    'Area': sample_df['Area'],
    'Predict_State': sample_df['Predict_State']
})

# Dodanie flagi pewności
comparison['Pewność'] = np.where(
    (comparison['Przewidziana Cena'] >= price_bounds[0.05]) & (comparison['Przewidziana Cena'] <= price_bounds[0.95]),
    'Wysoka',
    'Niska (Outlier?)'
)

display(comparison)


--- Ocena finalnego modelu na zbiorze walidacyjnym ---
Validation RMSE (on log scale): 0.2155

Model z warstwami preprocessingu zapisany w: price_regressor_v7_with_preprocessing.keras

Obliczanie efektywności procentowej...

--- Efektywność Procentowa Nowego Modelu ---
Średni Absolutny Błąd Procentowy (MAPE): 16.71%
Mediana Absolutnego Błędu Procentowego: 12.22%
Interpretacja: Dla połowy mieszkań błąd predykcji jest mniejszy niż 12.22%.

--- Test predykcji z filtrowaniem pewności ---
Uznajemy predykcje za 'pewne', jeśli mieszczą się w przedziale: 220,000 - 1,250,000 PLN
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step


,Prawdziwa Cena,Przewidziana Cena,Area,Predict_State,Pewność
873982,"219,000.00","310,808.22",39.00,FOR_RENOVATION,Wysoka
901142,"219,000.00","427,277.31",46.80,FOR_RENOVATION,Wysoka
451278,"549,000.00","636,918.12",41.00,GOOD,Wysoka
1323846,"1,000,000.00","1,186,514.25",60.00,GOOD,Wysoka
640117,"350,000.00","263,223.22",51.00,GOOD,Wysoka
1345409,"509,000.00","516,926.91",40.00,GOOD,Wysoka
756625,"549,000.00","477,491.84",37.71,FOR_RENOVATION,Wysoka
879408,"329,000.00","351,949.56",54.37,GOOD,Wysoka
68498,"399,000.00","357,029.28",39.00,DEVELOPER_STATE,Wysoka
279806,"135,000.00","214,196.64",36.25,AFTER_RENOVATION,Niska (Outlier?)
